In [7]:
#Modules to install via pip pandas,ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pprint import pprint
import os
import import_ipynb
import sys
import kmeans
sys.path.append('../')

import trace_analysis
import os
import trace_statistics 
from functions import *
from pandas.plotting import scatter_matrix
import cmath as math
from mpl_toolkits.mplot3d import Axes3D
from sklearn import cluster
from sklearn.metrics import confusion_matrix
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.cluster import KMeans
# scipy
from scipy.cluster.vq import kmeans,vq,whiten
import sklearn.metrics as sm
import seaborn as sns
from sklearn.decomposition import PCA
%matplotlib inline

In [9]:
# Read the ip of each node
ips = pd.read_csv('./traces/out-2019-01JAN-28-01.cap', 
                        sep=';|addr:|/',
                        na_filter=True,
                        usecols=[1,3,4], 
                        header=None, 
                        nrows=550,
                        names=['prefix','node_id','addr','ip','scope'],
                        engine='python').dropna()

ips = ips[ips.scope == '64  scope: global'].reset_index(drop=True).drop(['scope'], axis=1)


# Read the rank of each node
rank = pd.read_csv('./traces/out-2019-01JAN-28-01.cap', 
                        sep=';|\t|R: | \| OP: ',
                        na_filter=True,
                        usecols=[1,4], 
                        header=None, 
                        skiprows=550,
                        names=['node_id','rank'],
                        engine='python').dropna()

rank = rank[rank['rank'].apply(lambda x: x.isdigit())].reset_index(drop=True)


# Merge all data
node_ip_and_rank = pd.merge(ips, rank, how='inner').drop_duplicates()
#node_ip_and_rank

,node_id,ip,rank
0,m3-97,2001:db8::1711:6b10:65fb:a522,768
2,m3-102,2001:db8::1711:6b10:65f9:5d3a,768
4,m3-166,2001:db8::1711:6b10:65f6:ad06,512
5,m3-142,2001:db8::1711:6b10:65fb:5f36,768
6,m3-129,2001:db8::1711:6b10:65fa:5f06,768
7,m3-145,2001:db8::1711:6b10:65f6:5c06,512
8,m3-161,2001:db8::1711:6b10:65f9:5e06,512
9,m3-121,2001:db8::1711:6b10:65f8:ae16,768
10,m3-126,2001:db8::1711:6b10:65f9:b522,768
11,m3-169,2001:db8::1711:6b10:65fd:aa32,512


In [11]:
# Read the rank of each node
nodes = pd.read_csv('./traces/out-2019-01JAN-28-01.cap', 
                        sep=';|seq=| hop|time = |ms',
                        na_filter=True,
                        usecols=[1,3,5], 
                        header=None, 
                        skiprows=799,
                        names=['node_id','seq','rtt'],
                        engine='python').dropna().drop_duplicates()

nodes = nodes.sort_values(by=['node_id','seq'], ascending=True, na_position='first')


#
d_nodes = {}   # <node_id, DataFrame containing seq and rtt columns>
for n in nodes.index:
    if nodes['node_id'][n] in d_nodes:
        d_nodes[nodes['node_id'][n]] = d_nodes[nodes['node_id'][n]].append(pd.DataFrame({'seq': [int(nodes['seq'][n])], nodes['node_id'][n]: [nodes['rtt'][n]]}))
    else:
        d_nodes[nodes['node_id'][n]] = pd.DataFrame({'seq': [int(nodes['seq'][n])], nodes['node_id'][n]:[nodes['rtt'][n]]})

    
# 
nodes = pd.DataFrame([seq for seq in range(1,1001)], columns=['seq']).set_index('seq')
for node in d_nodes.keys():
    nodes = nodes.join(d_nodes[node].set_index('seq'))

nodes = nodes[~nodes.index.duplicated(keep='first')]
#nodes.head(10)

In [13]:
rank_to_hops = sorted([int(rank) for rank in list(node_ip_and_rank['rank'].drop_duplicates())])

hops = {}
icmp = [x for x in range(1,len(nodes)+1)]
for node in node_ip_and_rank.index:
    if (rank_to_hops.index(int(node_ip_and_rank['rank'][node]))+1) in hops:
        # The key should be created
        hops[rank_to_hops.index(int(node_ip_and_rank['rank'][node]))+1].append(node_ip_and_rank['node_id'][node])
    else:
        # Just append to the list of nodes
        hops[rank_to_hops.index(int(node_ip_and_rank['rank'][node]))+1] = [node_ip_and_rank['node_id'][node]]


# Contain mean time for each distance from the root
hop_nodes = pd.DataFrame({1: nodes[hops[1]].mean(axis=1), 2: nodes[hops[2]].mean(axis=1),\
                           3: nodes[hops[3]].mean(axis=1)})
#hop_nodes.head(10)

In [17]:
data=nodes.describe().T
data.head()


,count,mean,std,min,25%,50%,75%,max
m3-102,146.0,40.648856,27.489136,0.604,28.11125,42.4620,58.83200,113.806
m3-105,337.0,26.626659,21.778591,0.604,0.62900,27.4270,41.67800,91.857
m3-110,150.0,37.742773,24.072587,0.604,28.70450,40.9910,55.10000,93.527
m3-113,331.0,28.495459,22.308389,0.611,0.63400,28.6900,41.55350,103.777
m3-118,280.0,25.407775,20.481679,0.604,0.63300,27.1295,37.92825,82.594


In [18]:
hop_nodes2 = trace_analysis.process_iotlab_aggregated(os.getcwd() + '/traces/', '2019-01JAN-30-1b169')
std_values2, outliers2 = trace_analysis.separate_outliers(hop_nodes2)
tubling_packet_loss2 = trace_statistics.tumbling_packet_loss_per_hop(os.getcwd() + '/traces/', '2019-01JAN-30-1b169', window_size=10)

../trace_analysis.py:29: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  rank['rank'] = rank['rank'].convert_objects(convert_numeric=True)
